# 1. Introduction

O objetivo deste projeto é extrair alguns insights sobre a venda de imóveis na King County, Washington State, USA. Os dados consistem no histórico de casas vendidas entre Maio de 2014 e Maio de 2015. Para extrair tais insights, algumas hipóteses serão testadas.

- H1: Imóveis que possuem vista para água são 30% mais caros na média. 

- H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média. 

- H3: Imóveis sem porão possuem área total (sqft_lot) são 40% maiores do que os imóveis com porão.

- H4: O crescimento do preço dos imóveis YoY (Year Over Year) é de 10%

- H5: Imóveis com 3 banheiros tem um crescimento de MoM (Month over Month) de 15%.

# 2. Questão do Negócio

1. Quais são os imóveis que a House Rocket deveria comprar e por qual preço?
2. Qual o melhor momento para vendê-lo e por qual preço?

# 3. Entendimento do negócio 

## 3.1. Produto final

Serão entregues dois relatórios:

1. Relatório com as sugestões de compras de imóveis e valor recomendado.
2. Relatório com as sugestões de venda de um imóvel por um valor recomendado.

## 3.2. Ferramentas

- Python
- Pycharm
- Jupyter Notebook

# 4. Premissas do negócio

- Que coisiderações foram feitas ao se fazer as análises? Descrever aqui!!!!

# 5. Planejamento da solução


## 5.1. Quais são os imóveis que a House Rocket deveria comprar e por qual preço?
1. Coletar os dados no Kaggle
2. Agrupar os dados por região (zipcode)
3. Dentro de cada região, encontrar a mediana do preço dos imóveis
4. Imóveis que estão abaixo do preço mediano de cada região e que esteja, em boas condições serão comprados.
5. A planilha final terá a seguinte estrutura:

|Imóvel Cond | Região | Preço do Imóvel | Preço da Mediana | Condição | Status |
|------------|--------|-----------------|------------------|----------|--------|
|10330       |302349  |U\$ 450.000        |U\$ 500.000        |3         |Compra  |


## 5.2. Qual o melhor momento para vendê-lo e por qual preço?
1. Agrupar os imóveis por região (zipcode) e por sazonalidae (Summer, winter)
2. Dentro de cada região e sazonalidade, eu vou calcular a mediana de preço.
3. Condições de venda:
    + Se o preço da compra for maior que a mediana da região + sazonalidade.
        + O preço da venda será igual ao preço da compra + 10 %
    
    + Se o preço da compra for menor que a mediana da região + sazonalidade
        + O preço da venda será igual ao preço da compra + 30 %
    
- Exemplo da planilha de saída:

|Imóvel.Cod| Região|Temporada|Preço da Mediana |Preço da compra |Preço da venda|Lucro| Condição |
|----------| ----------|----------|----------|---------- |----------|----------| ---------- |
|10330 | 302349 | Verão | R$ 800.000,00 | R\$ 450.000,00 | R\$ 450.000,00 + 30\%| ??|

# 6. Execução da solução

## 6.1. Packages

In [121]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import seaborn as sns
import plotly.express as px
from statistics import mean

pd.set_option('display.float_format', lambda x: '%.3f' % x) # Modificar notação científica

## 6.2. Loading dataset

In [2]:
data = pd.read_csv('datasets/kc_house_data.csv')
data.shape

(21613, 21)

In [3]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
data.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [5]:
data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

## 6.3. Data Transformation

In [6]:
df = data.copy()
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Transforming the variable `date` type:


In [7]:
df['date'] = pd.to_datetime(df['date'])

### Creating `month` and `year` variables:


In [8]:
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

### Creating `season` variable:

Seasons dates can be checked in this website: https://www.calendardate.com/year2014.php

In [9]:
df['date'].min()

Timestamp('2014-05-02 00:00:00')

In [10]:
df['date'].max()

Timestamp('2015-05-27 00:00:00')

In [11]:
# test
(df['date'][0] >= pd.to_datetime('2014-03-20')) & (df['date'][0] < pd.to_datetime('2014-06-20'))

False

In [12]:
# Creating variable `season`
df['season'] = 0

In [54]:
# Populating `season` variable
df['season'] = df['date'].apply(lambda x: 'Spring' if ((x >= pd.to_datetime('2014-03-20')) & (x < pd.to_datetime('2014-06-21'))) | ((x >= pd.to_datetime('2015-03-20')) & (x < pd.to_datetime('2015-06-20'))) else
                                          'Summer' if (x >= pd.to_datetime('2014-06-21')) & (x < pd.to_datetime('2014-09-22'))  else
                                          'Fall' if (x >= pd.to_datetime('2014-09-22')) & (x < pd.to_datetime('2014-12-21')) else
                                          'Winter'
                                          )






In [48]:
# Creating variable `old`
df['old'] = 0
df['old'] = df['yr_built'].apply(lambda x: '< 1955' if x < 1955 else
                                          '>= 1955' 
                                          )


In [49]:
df['old'].value_counts()

>= 1955    15449
< 1955      6164
Name: old, dtype: int64

In [70]:
# Creating the variable `basement` 
df['basement'] = 0
df['basement'] = df['sqft_basement'].apply(lambda x: 'Yes' if x !=0 else
                                          'No' 
                                          )

In [71]:
df['basement'].value_counts()

No     13126
Yes     8487
Name: basement, dtype: int64

### Grouping by zipcode and calculating median price per zipcode

In [14]:
df_price_median = df[['price', 'zipcode']].groupby('zipcode').median().reset_index()
df_price_median

,zipcode,price
0,98001,260000.0
1,98002,235000.0
2,98003,267475.0
3,98004,1150000.0
4,98005,765475.0
...,...,...
65,98177,554000.0
66,98178,278277.0
67,98188,264000.0
68,98198,265000.0


In [15]:
df_price_median = df_price_median.rename(columns = {'price': 'median_price'})

In [16]:
# Merging data frames -----------
df1 = pd.merge(df, df_price_median, on='zipcode', how='inner')
df1

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,season,median_price
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,98178,47.5112,-122.257,1340,5650,10,2014,Fall,278277.0
1,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,0,98178,47.5009,-122.248,1190,6780,6,2014,Summer,278277.0
2,4058801670,2014-07-17,445000.0,3,2.25,2100,8201,1.0,0,2,...,0,98178,47.5091,-122.244,2660,8712,7,2014,Summer,278277.0
3,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,0,...,0,98178,47.5053,-122.255,1320,7619,9,2014,Fall,278277.0
4,6874200960,2015-02-27,170000.0,2,1.00,860,5265,1.0,0,0,...,0,98178,47.5048,-122.272,1650,8775,2,2015,Winter,278277.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,2525049086,2014-10-03,2720000.0,4,3.25,3990,18115,2.0,0,0,...,0,98039,47.6177,-122.229,3450,16087,10,2014,Fall,1892500.0
21609,2525049113,2014-07-25,1950000.0,4,3.50,4065,18713,2.0,0,0,...,0,98039,47.6209,-122.237,3070,18713,7,2014,Summer,1892500.0
21610,3262300485,2015-04-21,2250000.0,5,5.25,3410,8118,2.0,0,0,...,0,98039,47.6295,-122.236,3410,16236,4,2015,Spring,1892500.0
21611,6447300365,2014-11-13,2900000.0,5,4.00,5190,14600,2.0,0,1,...,0,98039,47.6102,-122.225,3840,19250,11,2014,Fall,1892500.0


## 6.4. First table: suggestions for buying

In [17]:
# Creating Status Variable -----------
df1['status'] = 0

for i in range(len(df1)):
    if(df1.loc[i, 'price'] < df1.loc[i, 'median_price']) & (df1.loc[i, 'condition'] >= 2):
        df1.loc[i, 'status'] = "buy"
    else:
        df1.loc[i, 'status'] = "do not buy"

In [18]:
df1

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,season,median_price,status
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,1340,5650,10,2014,Fall,278277.0,buy
1,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,98178,47.5009,-122.248,1190,6780,6,2014,Summer,278277.0,buy
2,4058801670,2014-07-17,445000.0,3,2.25,2100,8201,1.0,0,2,...,98178,47.5091,-122.244,2660,8712,7,2014,Summer,278277.0,do not buy
3,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,0,...,98178,47.5053,-122.255,1320,7619,9,2014,Fall,278277.0,buy
4,6874200960,2015-02-27,170000.0,2,1.00,860,5265,1.0,0,0,...,98178,47.5048,-122.272,1650,8775,2,2015,Winter,278277.0,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,2525049086,2014-10-03,2720000.0,4,3.25,3990,18115,2.0,0,0,...,98039,47.6177,-122.229,3450,16087,10,2014,Fall,1892500.0,do not buy
21609,2525049113,2014-07-25,1950000.0,4,3.50,4065,18713,2.0,0,0,...,98039,47.6209,-122.237,3070,18713,7,2014,Summer,1892500.0,do not buy
21610,3262300485,2015-04-21,2250000.0,5,5.25,3410,8118,2.0,0,0,...,98039,47.6295,-122.236,3410,16236,4,2015,Spring,1892500.0,do not buy
21611,6447300365,2014-11-13,2900000.0,5,4.00,5190,14600,2.0,0,1,...,98039,47.6102,-122.225,3840,19250,11,2014,Fall,1892500.0,do not buy


In [19]:
# Creating Dataframe to be delivered -----------
df2 = df1[['id', 'zipcode', 'price', 'median_price', 'condition', 'status']].copy()
df2.columns = ['ID', 'Region', 'Price', 'Median Price', 'Condition', 'Status']
df2.head()

,ID,Region,Price,Median Price,Condition,Status
0,7129300520,98178,221900.0,278277.0,3,buy
1,4060000240,98178,205425.0,278277.0,4,buy
2,4058801670,98178,445000.0,278277.0,3,do not buy
3,2976800796,98178,236000.0,278277.0,3,buy
4,6874200960,98178,170000.0,278277.0,3,buy


## 6.5.  Second Table: Price suggestions for buying and selling

- Here we are going to consider only the properties with the Status equal to "buy" in this analysis. 

In [20]:
df3 = df1[df1['status'] == "buy"].copy().reset_index()
df3

,index,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,season,median_price,status
0,0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,...,98178,47.5112,-122.257,1340,5650,10,2014,Fall,278277.0,buy
1,1,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,...,98178,47.5009,-122.248,1190,6780,6,2014,Summer,278277.0,buy
2,3,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,...,98178,47.5053,-122.255,1320,7619,9,2014,Fall,278277.0,buy
3,4,6874200960,2015-02-27,170000.0,2,1.00,860,5265,1.0,0,...,98178,47.5048,-122.272,1650,8775,2,2015,Winter,278277.0,buy
4,5,4268200055,2015-05-01,245000.0,3,1.75,1740,11547,1.0,0,...,98178,47.4945,-122.220,880,78408,5,2015,Spring,278277.0,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10708,21600,3025300225,2014-10-31,1450000.0,5,2.75,3090,19865,1.0,0,...,98039,47.6232,-122.235,2970,19862,10,2014,Fall,1892500.0,buy
10709,21601,2470200020,2014-05-14,1880000.0,4,2.75,3260,19542,1.0,0,...,98039,47.6245,-122.236,3480,19863,5,2014,Spring,1892500.0,buy
10710,21603,3625049079,2014-08-01,1350000.0,3,2.00,2070,9600,1.0,0,...,98039,47.6160,-122.239,3000,16215,8,2014,Summer,1892500.0,buy
10711,21605,5427100150,2014-06-26,1410000.0,4,2.25,3250,16684,2.0,0,...,98039,47.6334,-122.229,2890,16927,6,2014,Summer,1892500.0,buy


### Grouping by zipcode and season

In [21]:
df4 = df3[['price', 'zipcode', 'season']].groupby(['zipcode', 'season']).median().reset_index()
df4.columns = ['zipcode', 'season', 'median_price_zipcode_season']
df4

,zipcode,season,median_price_zipcode_season
0,98001,Fall,215000.0
1,98001,Spring,214550.0
2,98001,Summer,215000.0
3,98001,Winter,218500.0
4,98002,Fall,189500.0
...,...,...,...
275,98198,Winter,205475.0
276,98199,Fall,525000.0
277,98199,Spring,540000.0
278,98199,Summer,508000.0


In [22]:
# Creating Sale price Variable -----------
df3['sale_price'] = 0

for i in range(len(df3)):
    if df3.loc[i, 'price'] >= float(df4[(df4['zipcode'] == df3['zipcode'][i]) & (df4['season'] == df3['season'][i])]['median_price_zipcode_season']):
        df3.loc[i, 'sale_price'] = 1.1* df3['price'][i]
    else:
        df3.loc[i, 'sale_price'] = 1.3* df3['price'][i]
        

In [23]:
# Calculating the profit
df3['profit'] = df3['sale_price'] - df3['price']

In [24]:
df3['profit'].sum()

760693197.9000001

In [25]:
df3

,index,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,long,sqft_living15,sqft_lot15,month,year,season,median_price,status,sale_price,profit
0,0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,...,-122.257,1340,5650,10,2014,Fall,278277.0,buy,244090.0,22190.0
1,1,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,...,-122.248,1190,6780,6,2014,Summer,278277.0,buy,267052.5,61627.5
2,3,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,...,-122.255,1320,7619,9,2014,Fall,278277.0,buy,259600.0,23600.0
3,4,6874200960,2015-02-27,170000.0,2,1.00,860,5265,1.0,0,...,-122.272,1650,8775,2,2015,Winter,278277.0,buy,221000.0,51000.0
4,5,4268200055,2015-05-01,245000.0,3,1.75,1740,11547,1.0,0,...,-122.220,880,78408,5,2015,Spring,278277.0,buy,269500.0,24500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10708,21600,3025300225,2014-10-31,1450000.0,5,2.75,3090,19865,1.0,0,...,-122.235,2970,19862,10,2014,Fall,1892500.0,buy,1595000.0,145000.0
10709,21601,2470200020,2014-05-14,1880000.0,4,2.75,3260,19542,1.0,0,...,-122.236,3480,19863,5,2014,Spring,1892500.0,buy,2068000.0,188000.0
10710,21603,3625049079,2014-08-01,1350000.0,3,2.00,2070,9600,1.0,0,...,-122.239,3000,16215,8,2014,Summer,1892500.0,buy,1485000.0,135000.0
10711,21605,5427100150,2014-06-26,1410000.0,4,2.25,3250,16684,2.0,0,...,-122.229,2890,16927,6,2014,Summer,1892500.0,buy,1551000.0,141000.0


# 7. Testing the hypothesis

## H1: Imóveis que possuem vista para água são 30% mais caros na média.

In [30]:
water_front_df = df[['price', 'waterfront']].groupby('waterfront').mean().reset_index()
water_front_df

,waterfront,price
0,0,531563.600
1,1,1661876.025


## Calculating the diference on the mean:

In [43]:
def diff_mean(val1, val2):
    percent = round(100*(val2 - val1)/val1,2)
    return percent

In [44]:
val1 = df[df['waterfront']==0]['price'].mean()
val2 = df[df['waterfront']==1]['price'].mean()
diff_mean(val1, val2)

212.64

## Plotting

In [61]:
import plotly.graph_objects as go

y0 = df[df['waterfront']==0]['price']
y1 = df[df['waterfront']==1]['price']

fig = go.Figure()
fig.add_trace(go.Box(y=y0, name='Not Waterfront',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=y1, name = 'Waterfront',
                marker_color = 'lightseagreen'))

fig.show()

- **Conclusion**: False. In average, waterfront properties are 212.64\% more expensive than the others. The mean price of waterfront proporties is `U$` 1,661,876.03 while the mean price of properties with no waterfront is `U$` 531,563.60

## H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.

In [50]:
df[['price', 'old']].groupby('old').mean().reset_index()

,old,price
0,< 1955,537050.908
1,>= 1955,541299.968


In [51]:
val1 = df[df['old']=='< 1955']['price'].mean()
val2 = df[df['old']!='< 1955']['price'].mean()
diff_mean(val1, val2)

0.79

In [69]:
import plotly.graph_objects as go
import numpy as np

y0 = df[df['old']=='< 1955']['price']
y1 = df[df['old']!='< 1955']['price']

fig = go.Figure()
fig.add_trace(go.Box(y=y0, name='< 1955',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=y1, name = '≥ 1955',
                marker_color = 'lightseagreen'))
fig.update_layout(
    title="Prices per Year built",
    xaxis_title="Year built",
    yaxis_title="Price"
)

fig.show()

- **Conclusion**: False. In average, "new" properties (year built >= 1955) are only 0.79\% more expensive than the old ones (year built < 1955). The mean price of the old proporties is `U$` 537,050.91 while the mean price of new properties is `U$` 541,299.97.

## H3: Imóveis sem porão possuem área total (sqft_lot) que são 40% maiores do que os imóveis com porão.

In [72]:
df[['sqft_lot', 'basement']].groupby('basement').mean().reset_index()

,basement,sqft_lot
0,No,16284.177
1,Yes,13286.295


In [76]:
val1 = df[df['basement']=='Yes']['sqft_lot'].mean()
val2 = df[df['basement']=='No']['sqft_lot'].mean()
diff_mean(val1, val2)

22.56

In [77]:
y0 = df[df['basement']=='Yes']['sqft_lot']
y1 = df[df['basement']=='No']['sqft_lot']
fig = go.Figure()
fig.add_trace(go.Box(y=y0, name='Yes',
                marker_color = 'indianred'))
fig.add_trace(go.Box(y=y1, name = 'No',
                marker_color = 'lightseagreen'))
fig.update_layout(
    title="Area per Basement",
    xaxis_title="Basement",
    yaxis_title="Area (sqft)"
)

fig.show()

- **Conclusion**: False. Although the proporties without basement are slightly bigger, they are only 22.56% bigger, not 40%. In average, properties without basement has area 16,284.18 square feet. Properties with basement has an area, in average, of  13,286.30 square feet. 

## H4: O crescimento do preço dos imóveis YoY (Year Over Year) é de 10%



In [78]:
df[['price', 'year']].groupby('year').mean().reset_index()

,year,price
0,2014,539181.428
1,2015,541988.992


In [93]:
mean_price_2014 = df[df['year']==2014]['price'].mean()
mean_price_2015 = df[df['year']==2015]['price'].mean()
print(diff_mean(mean_price_2014, mean_price_2015))

# Standard deviation
std_price_2014 = df[df['year']==2014]['price'].std()
std_price_2015 = df[df['year']==2015]['price'].std()


0.52


In [92]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    name='2014',
    x=['Year'], y=[mean_price_2014],
    error_y=dict(type='data', array=[std_price_2014])
))
fig.add_trace(go.Bar(
    name='2015',
    x=['Year'], y=[mean_price_2015],
    error_y=dict(type='data', array=[std_price_2015])
))
fig.update_layout(barmode='group')
fig.show()

- **Conclusion**: False. The YOY growth was only 0.52% between 2014 and 2015. 

## H5: Imóveis com 3 banheiros tem um crescimento de preço MoM (Month over Month) de 15%.

- Filtrar os imóveis com 3 banheiros e salvar na base de dados: three_bathrooms_df
- Calcular a média de preço da base de dados three_bathrooms_df agrupada por meses
- Calcular os crescimentos MoM
- Criar uma visualização de linhas e pontos mostrando o crescimento por mês

In [105]:
three_bathrooms_df = df[df['bathrooms']==3][['price', 'month']].groupby('month').mean().reset_index()
print(three_bathrooms_df)

    month      price
0       1 672923.171
1       2 696714.422
2       3 768495.096
3       4 709751.988
4       5 689252.100
5       6 775057.564
6       7 741405.043
7       8 737576.562
8       9 645246.887
9      10 645523.015
10     11 679910.465
11     12 728341.188


In [113]:
i = 1
three_bathrooms_df.loc[i-1,'price']

672923.1707317074

In [119]:
# Calculating MoM Growth
MoM=[]
for i in range(1,11):
    val1=three_bathrooms_df.loc[i-1,'price']
    val2=three_bathrooms_df.loc[i,'price']
    result=diff_mean(val1, val2)
    MoM.append(result) 
print(MoM)

[3.54, 10.3, -7.64, -2.89, 12.45, -4.34, -0.52, -12.52, 0.04, 5.33]


In [122]:
# Mean MoM Growth
mean(MoM)

0.3750000000000001

In [109]:
import plotly.express as px
fig = px.line(three_bathrooms_df, x='month', y='price')
fig.show()

- **Conclusion**: False. The MoM growth was only 0.375%. 

## H6: Imóveis são reformados, em média, após 30 anos da data de construção.

In [131]:
renovated_df = df[df['yr_renovated'] != 0].copy().reset_index()
renovated_df['time_diff'] = renovated_df['yr_renovated'] - renovated_df['yr_built']
mean(renovated_df['time_diff'])

56.29978118161926

In [130]:
fig = px.histogram(renovated_df, x="time_diff")
fig.show()

- **Conclusion**: False. Properties are renovated, in average, 56.3 years after they are built.

## H7: A área dos imóveis YoY (Year Over Year) é 5% menor. 

## H8: Os imóveis são 20%  mais baratos no inverno.

## H9: No verão o lucro de vendas é 35% maior que no inverno.

## H10: A área de imóveis com waterfront é 25% maior que a área dos imóveis sem waterfront.

# 8. Mapa com as sugestões de imóveis para compra

## 8.1. Filtros

- Número de quartos
- Número de banheiros
- Andares
- Waterfront
- Preço